In [1]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [2]:
# Mount Google Drive and set data paths.
import os
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# ML4H - Project 2

## Task 1

In [3]:
# Gloabal variables
data_path = "/content/gdrive/My Drive/ETH/MLH/Project2/data/PubMed_200k_RCT/"
model_path = "/content/gdrive/My Drive/ETH/MLH/Project1/model/"

seed = 50

In [4]:
from tqdm.notebook import tqdm

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
train_df = pd.read_csv(data_path + 'pre_processed_train.csv', index_col=0)
test_df = pd.read_csv(data_path + 'pre_processed_test.csv', index_col=0)
dev_df = pd.read_csv(data_path + 'pre_processed_dev.csv', index_col=0)

In [14]:
train_df.head(5)

,Label,Words
0,BACKGROUND,"['emerg', 'hiv', 'chronic', 'condit', 'mean', ..."
1,BACKGROUND,"['paper', 'describ', 'design', 'evalu', 'posit..."
2,METHODS,"['studi', 'design', 'randomis', 'control', 'tr..."
3,METHODS,"['intervent', 'group', 'particip', 'onlin', 'g..."
4,METHODS,"['program', 'base', 'self', 'efficaci', 'theor..."


In [15]:
train_lab = train_df.pop('Label')
train_words = train_df.pop('Words')

test_lab = test_df.pop('Label')
test_words = test_df.pop('Words')

dev_lab = dev_df.pop('Label')
dev_words = dev_df.pop('Words')

In [16]:
train_lab[0]

'BACKGROUND'

In [17]:
train_words[0]

"['emerg', 'hiv', 'chronic', 'condit', 'mean', 'peopl', 'live', 'hiv', 'requir', 'take', 'respons', 'self', 'manag', 'condit', 'includ', 'make', 'physic', 'emot', 'social', 'adjust']"

In [19]:
def dummy_fun(doc):
    return doc

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)  

In [ ]:
tfidf.fit(train_words)